In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 13:35:56 2020

@author: wanghaochen
"""

import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

path = './Dataset/'
Fullevents = pd.read_csv(path + 'fullevents.csv')
data_for_Pscore = pd.read_csv(path + 'fullevents.csv')

data_for_Pscore_byOpponents = pd.read_csv(path + 'fullevents.csv')
Passingevents = pd.read_csv(path + 'passingevents.csv')
# 按比赛计算 Pezzali Score(team)
huskies_goals = list(data_for_Pscore['OwnScore'])
huskies_attempts = list(data_for_Pscore['attempts'])
huskies_points = list(data_for_Pscore['points'])
opponent_goals = list(data_for_Pscore['OpponentScore'])
opponent_attempts = list(data_for_Pscore['op_attempts'])
opponent_points = list(data_for_Pscore['op_points'])

huskies_goals_std = (huskies_goals - np.mean(huskies_goals)) / np.std(huskies_goals)
huskies_attempts_std = (huskies_attempts - np.mean(huskies_attempts)) / np.std(huskies_attempts)
opponent_goals_std = (opponent_goals - np.mean(opponent_goals)) / np.std(opponent_goals)
opponent_attempts_std = (opponent_attempts - np.mean(opponent_attempts)) / np.std(opponent_attempts)

p_score_huskies = np.abs(huskies_goals_std) / np.abs(huskies_attempts_std)*np.abs(opponent_attempts_std) / np.abs(opponent_goals_std)
p_score_huskies = list(p_score_huskies)

p_score_hus_attackEffcncy = list(np.abs(huskies_goals_std) / np.abs(huskies_attempts_std))
p_score_hus_defenceEffcncy = list(np.abs(opponent_goals_std) / np.abs(opponent_attempts_std))

p_score_df = pd.DataFrame({'p_score': p_score_huskies,
                           'p_attack': p_score_hus_attackEffcncy,
                           'p_defence': p_score_hus_defenceEffcncy})

p_score_huskies_mean = np.mean(p_score_huskies)  # 2.744461088567615
# 绘图
_matchId = [int(i) for i in data_for_Pscore['MatchID']]
plt.figure(dpi=100)
plt.plot(_matchId, p_score_huskies)
plt.xlabel('MatchID')
plt.ylabel('Pezzali Score')
plt.grid()
plt.show()

# 按比赛对手计算 Pezzali Score(team)
huskies_goals_op = list(data_for_Pscore_byOpponents['OwnScore'])
huskies_attempts_op = list(data_for_Pscore_byOpponents['attempts'])
huskies_points_op = list(data_for_Pscore_byOpponents['points'])
opponent_goals_op = list(data_for_Pscore_byOpponents['OpponentScore'])
opponent_attempts_op = list(data_for_Pscore_byOpponents['op_attempts'])
opponent_points_op = list(data_for_Pscore_byOpponents['op_points'])

huskies_goals_op_std = (huskies_goals_op - np.mean(huskies_goals_op)) / np.std(huskies_goals_op)
huskies_attempts_op_std = (huskies_attempts_op - np.mean(huskies_attempts_op)) / np.std(huskies_attempts_op)
opponent_goals_op_std = (opponent_goals_op - np.mean(opponent_goals_op)) / np.std(opponent_goals_op)
opponent_attempts_op_std = (opponent_attempts_op - np.mean(opponent_attempts_op)) / np.std(opponent_attempts_op)

p_score_huskies_op = np.abs(huskies_goals_op_std) / np.abs(huskies_attempts_op_std)*np.abs(opponent_attempts_op_std) / np.abs(opponent_goals_op_std)
p_score_huskies_op = list(p_score_huskies_op)

p_score_huskies_op_mean = np.mean(p_score_huskies_op)  # 2.744461088567615

# 绘图
_opID = list(data_for_Pscore_byOpponents['OpponentID'])
_n = len(_opID)
_opID = list(np.linspace(1, _n, _n))
_opID = [str(int(i)) for i in _opID]
plt.figure(dpi=100)
plt.plot(_opID, p_score_huskies_op)
plt.xlabel('MatchID')
plt.ylabel('Pezzali Score')
plt.grid()
plt.show()

# 分场次球员贡献，计算各个event各球员的发生次数占比，求调和平均得到球员贡献
EventType = list(set(Fullevents['EventType']))
EventSubType = list(set(Fullevents['EventSubType']))
'''
'Foul'（犯规）, 'Duel'（硬碰硬）, 'Offside'（越位）, 'Shot'（射门）, 'Goalkeeper leaving line',
'Save attempt','Pass', 'Others on the ball', 'Free Kick', 'Substitution', 'Interruption'
'''

# 确定各EventType权重
set_weight = {'Foul': -2, 'Duel': 3, 'Offside': -1, 'Shot': 3, 'Goalkeeper leaving line': 1,
              'Save attempt': 2, 'Pass': 1, 'Others on the ball': 1, 'Free Kick': 2,
              'Substitution': 0, 'Interruption': -1}
_df = Fullevents[Fullevents['TeamID'] == 'Huskies']
huskies_player = list(set(_df['OriginPlayerID']))
match_ID = list(set(Fullevents['MatchID']))
n_player = len(huskies_player)
n_match = len(match_ID)
match_ID = [str(i) for i in range(1, n_match + 1)]
huskies_contribution = np.zeros([n_player, n_match])

for k in range(n_match):
    _df0 = Fullevents[Fullevents['MatchID'] == match_ID[k]]
    for i in range(n_player):
        _df = _df0[_df0['OriginPlayerID'] == huskies_player[i]]
        for j in range(_df.shape[0]):
            _eventType = _df['EventType'].iloc[j]
            huskies_contribution[i, k] += set_weight[_eventType]

# 归一化
_min = np.min(huskies_contribution)
_max = np.max(huskies_contribution)
huskies_contribution = (np.asfarray(huskies_contribution) - _min) / (_max - _min)

# pd.DataFrame
contrib_distrib = pd.DataFrame(huskies_contribution, index=huskies_player, columns=match_ID)

# 绘图
for i in range(contrib_distrib.shape[1]):
    plt.figure()
    plt.title('MatchID: ' + str(i + 1))
    plt.ylabel('Contribution')
    plt.xlabel('PlayerID')
    contrib_distrib.iloc[:, i].plot('bar')
    plt.show()


# 分场次球员贡献描述性统计分析
def stats_describe(var_list):
    max_ = np.max(var_list)  # 最大值
    mean = np.mean(var_list)  # 平均值
    std = np.std(var_list)  # 标准差
    var_stats = [max_, mean, std]
    return var_stats


contrib_stats = []
for i in range(contrib_distrib.shape[1]):
    contrib_stats.append(stats_describe(contrib_distrib.iloc[:, i]))
_idx = np.linspace(1, n_match, n_match)
_col = ['Max', 'Mean', 'Std']
contrib_stats = pd.DataFrame(contrib_stats, index=_idx, columns=_col)
plt.figure()
plt.plot(_idx, contrib_stats['Max'])
plt.plot(_idx, contrib_stats['Mean'])
plt.plot(_idx, contrib_stats['Std'])
plt.xlim([1, 38])
plt.legend(_col)
plt.grid()
plt.show()
# 分场次比赛策略，进攻为主--2，攻防兼备--1，防守为主--0
match_ID = list(set(Fullevents['MatchID']))
n_match = len(match_ID)
match_ID = [str(i) for i in range(1, n_match + 1)]

strategy = np.zeros([n_match, 2])

for i in range(n_match):
    _df = Fullevents[Fullevents['MatchID'] == str(i + 1)]
    _df_hus = _df[_df['TeamID'] == 'Huskies']
    _df_opp = _df[_df['TeamID'] != 'Huskies']
    _df_hus = _df_hus[_df_hus['EventType'] != 'Substitution']
    _df_opp = _df_opp[_df_opp['EventType'] != 'Substitution']
    _x_hus_1 = list(_df_hus['EventOrigin_x'])
    _y_hus_1 = list(_df_hus['EventOrigin_y'])
    _x_opp_1 = list(_df_opp['EventOrigin_x'])
    _y_opp_1 = list(_df_opp['EventOrigin_y'])

    _df_hus = _df_hus.dropna()
    _df_opp = _df_opp.dropna()
    _x_hus_2 = list(_df_hus['EventDestination_x'])
    _y_hus_2 = list(_df_hus['EventDestination_y'])
    _x_opp_2 = list(_df_opp['EventDestination_x'])
    _y_opp_2 = list(_df_opp['EventDestination_y'])

    _x_hus = _x_hus_1 + _x_hus_2
    _y_hus = _y_hus_1 + _y_hus_2
    _x_opp = _x_opp_1 + _x_opp_2
    _y_opp = _y_opp_1 + _y_opp_2

    _x_opp = 100 - np.asfarray(_x_opp)
    _y_opp = 100 - np.asfarray(_y_opp)
    _mean_x_hus = np.average(_x_hus)
    _mean_x_opp = np.average(_x_opp)

    if _mean_x_hus > 52:
        strategy[i, 0] = 2
    elif _mean_x_hus > 48:
        strategy[i, 0] = 1
    else:
        strategy[i, 0] = 0

    if _mean_x_opp > 52:
        strategy[i, 1] = 2
    elif _mean_x_opp > 48:
        strategy[i, 1] = 1
    else:
        strategy[i, 1] = 0

    print(np.average(_x_hus), np.average(_x_opp))
    plt.figure()
    plt.title('MatchID: ' + str(i + 1))
    plt.ylabel('Y')
    plt.xlabel('X')
    plt.plot(_x_hus, _y_hus, 'b.')
    plt.plot(_x_opp, _y_opp, 'r.')
    plt.show()

strategy = pd.DataFrame(strategy, index=match_ID, columns=['Huskies', 'Opponents'])
# 绘图 平均球员位置
plt.figure()
# strategy.plot()
strategy['Huskies'].plot()
# strategy['Opponents'].plot()
plt.xlabel('MatchID')
plt.ylabel('average_x')
plt.show()

# 记录射门时点
ShotTime = Fullevents[Fullevents['EventType'] == 'Shot']
ShotTime_hus = ShotTime[ShotTime['TeamID'] == 'Huskies']
ShotTime_opp = ShotTime[ShotTime['TeamID'] != 'Huskies']
ShotTime_hus_index = list(ShotTime_hus.index)
ShotTime_opp_index = list(ShotTime_opp.index)

# 对手进攻
BeforeShot_opp = []
for i in range(len(ShotTime_opp_index)):
    _df = Fullevents.iloc[ShotTime_opp_index[i] - 10: ShotTime_opp_index[i] + 1, :]
    BeforeShot_opp.append(_df)
# 绘制每一次进攻，对手的进攻路线图
for i in range(len(BeforeShot_opp)):
    _df = BeforeShot_opp[i]
    # plt.figure(dpi=100)
    fig, ax = plt.subplots()
    for j in range(_df.shape[0] - 1):
        _x = [_df['EventOrigin_x'].iloc[j], _df['EventDestination_x'].iloc[j]]
        _y = [_df['EventOrigin_y'].iloc[j], _df['EventDestination_y'].iloc[j]]

        if _df['TeamID'].iloc[j] == 'Huskies':
            _x[0] = 100 - _x[0]
            _x[1] = 100 - _x[1]
            _y[0] = 100 - _y[0]
            _y[1] = 100 - _y[1]
            ax.plot(_x, _y, 'b.')
            ax.plot(_x, _y, 'b-', linewidth=0.5)
        else:
            ax.plot(_x, _y, 'r.')
            ax.plot(_x, _y, 'r-', linewidth=0.5)
    ax.plot(_df['EventOrigin_x'].iloc[j], _df['EventOrigin_y'].iloc[j], 'r.', label='Opponents')
    ax.legend()
    plt.title('MatchID:' + _df['MatchID'].iloc[0] + ' || Opponents Attacking')
    plt.xlabel("The opponent's goal is at the origin of the x axis")
    plt.show()
# 哈士奇进攻
BeforeShot_hus = []
for i in range(len(ShotTime_hus_index)):
    _df = Fullevents.iloc[ShotTime_hus_index[i] - 10: ShotTime_hus_index[i] + 1, :]
    BeforeShot_hus.append(_df)
# 绘制每一次进攻，哈士奇的进攻路线图
for i in range(len(BeforeShot_hus)):
    _df = BeforeShot_hus[i]
    # plt.figure(dpi=100)
    fig, ax = plt.subplots()
    for j in range(_df.shape[0] - 1):
        _x = [_df['EventOrigin_x'].iloc[j], _df['EventDestination_x'].iloc[j]]
        _y = [_df['EventOrigin_y'].iloc[j], _df['EventDestination_y'].iloc[j]]

        if _df['TeamID'].iloc[j] == 'Huskies':
            ax.plot(_x, _y, 'b.')
            ax.plot(_x, _y, 'b-', linewidth=0.5)
        else:
            _x[0] = 100 - _x[0]
            _x[1] = 100 - _x[1]
            _y[0] = 100 - _y[0]
            _y[1] = 100 - _y[1]
            ax.plot(_x, _y, 'r.')
            ax.plot(_x, _y, 'r-', linewidth=0.5)
    plt.plot(_df['EventOrigin_x'].iloc[j], _df['EventOrigin_y'].iloc[j], 'b.', label='Huskies')
    ax.legend()
    plt.title('MatchID:' + _df['MatchID'].iloc[0] + ' || Huskies Attacking')
    plt.xlabel("The Huskies's goal is at the origin of the x axis")
    plt.show()
# 对手射门前后四个event
DuringShot_opp = []
for i in range(len(ShotTime_opp_index) - 1):
    _df = Fullevents.iloc[ShotTime_opp_index[i] - 1: ShotTime_opp_index[i] + 3, :]
    _df.reindex([0, 1, 2, 3])
    DuringShot_opp.append(_df)
# 哈士奇射门前后四个event
DuringShot_hus = []
for i in range(len(ShotTime_hus_index)):
    _df = Fullevents.iloc[ShotTime_hus_index[i] - 1: ShotTime_hus_index[i] + 3, :]
    _df.reindex([0, 1, 2, 3])
    DuringShot_hus.append(_df)
# 对进攻做聚类分析，聚为两类。分为成功与失败，对手进攻成功即为哈士奇防守失败
# 9各变量 除射门外各EventType、EventSubType，射门xy坐标，起止时间差
_c = ['et_1', 'et_2', 'et_3', 'est_1', 'est_2', 'est_3', 'shot_x', 'shot_y', 'duration']
DuringShot_opp_mat = np.zeros([len(DuringShot_opp), 9])
DuringShot_hus_mat = np.zeros([len(DuringShot_hus), 9])
EventType = list(set(Fullevents['EventType']))
EventSubType = list(set(Fullevents['EventSubType']))

EventType_dict = {}
EventSubType_dict = {}
for i in range(len(EventType)):
    EventType_dict[EventType[i]] = i + 1
for i in range(len(EventSubType)):
    EventSubType_dict[EventSubType[i]] = i + 1

for i in range(len(DuringShot_opp)):
    _df = DuringShot_opp[i]
    DuringShot_opp_mat[i, 0] = EventType_dict[_df['EventType'].iloc[0]]
    DuringShot_opp_mat[i, 1] = EventType_dict[_df['EventType'].iloc[2]]
    DuringShot_opp_mat[i, 2] = EventType_dict[_df['EventType'].iloc[3]]
    DuringShot_opp_mat[i, 3] = EventSubType_dict[_df['EventSubType'].iloc[0]]
    DuringShot_opp_mat[i, 4] = EventSubType_dict[_df['EventSubType'].iloc[2]]
    DuringShot_opp_mat[i, 5] = EventSubType_dict[_df['EventSubType'].iloc[3]]
    DuringShot_opp_mat[i, 6] = _df['EventTime'].iloc[3] - _df['EventTime'].iloc[0]
    DuringShot_opp_mat[i, 7] = _df['EventOrigin_x'].iloc[3]
    DuringShot_opp_mat[i, 8] = _df['EventOrigin_y'].iloc[3]
    if DuringShot_opp_mat[i, 6] < 0:
        DuringShot_opp_mat[i, 6] = 60
    if np.isnan(np.min(DuringShot_opp_mat[:, 7])):
        DuringShot_opp_mat[i, 7] = 100
        DuringShot_opp_mat[i, 8] = 100
DuringShot_opp_mat = np.asfarray(DuringShot_opp_mat)
DuringShot_opp_mat = pd.DataFrame(DuringShot_opp_mat, columns=_c)

for i in range(len(DuringShot_hus)):
    _df = DuringShot_hus[i]
    DuringShot_hus_mat[i, 0] = EventType_dict[_df['EventType'].iloc[0]]
    DuringShot_hus_mat[i, 1] = EventType_dict[_df['EventType'].iloc[2]]
    DuringShot_hus_mat[i, 2] = EventType_dict[_df['EventType'].iloc[3]]
    DuringShot_hus_mat[i, 3] = EventSubType_dict[_df['EventSubType'].iloc[0]]
    DuringShot_hus_mat[i, 4] = EventSubType_dict[_df['EventSubType'].iloc[2]]
    DuringShot_hus_mat[i, 5] = EventSubType_dict[_df['EventSubType'].iloc[3]]
    DuringShot_hus_mat[i, 6] = _df['EventTime'].iloc[3] - _df['EventTime'].iloc[0]
    DuringShot_hus_mat[i, 7] = _df['EventOrigin_x'].iloc[3]
    DuringShot_hus_mat[i, 8] = _df['EventOrigin_y'].iloc[3]
    if DuringShot_hus_mat[i, 6] < 0:
        DuringShot_hus_mat[i, 6] = 60
    if np.isnan(np.min(DuringShot_hus_mat[:, 7])):
        DuringShot_hus_mat[i, 7] = 100
        DuringShot_hus_mat[i, 8] = 100
DuringShot_hus_mat = np.asfarray(DuringShot_hus_mat)
DuringShot_hus_mat = pd.DataFrame(DuringShot_hus_mat, columns=_c)

# 对手进攻哈士奇防守聚类分析
#设置类别为2
clf_opp = KMeans(n_clusters=2)
#将数据代入到聚类模型中
clf_opp = clf_opp.fit(DuringShot_opp_mat)
clf_opp.cluster_centers_
#在原始数据表中增加聚类结果标签
DuringShot_opp_mat['label'] = clf_opp.labels_
# 聚类失败
# 球员分析
# 球员接球后的反应
_df = Fullevents[Fullevents['TeamID'] == 'Huskies']
huskies_player = list(set(_df['OriginPlayerID']))
match_ID = list(set(Fullevents['MatchID']))
n_player = len(huskies_player)
n_match = len(match_ID)
match_ID = [str(i) for i in range(1, n_match + 1)]

# 接球时间
player_catchBallTime = []
for i in range(n_player):
    _df = Fullevents[Fullevents['DestinationPlayerID'] == huskies_player[i]]
    player_catchBallTime.append(list(_df.index))

player_catchBall = []
for i in range(n_player):
    _idx = player_catchBallTime[i][0]
    _df = Fullevents.iloc[_idx:_idx + 1, :]
    for j in range(1, len(player_catchBallTime[i])):
        _idx = player_catchBallTime[i][j]
        _df = pd.concat([_df, Fullevents.iloc[_idx:_idx + 1, :]])
    player_catchBall.append(_df)

# 接球之后的一个event
AfterPlayerCatchBall = []
for i in range(n_player):
    _idx = player_catchBallTime[i][0]
    _df = Fullevents.iloc[_idx + 1:_idx + 2, :]
    for j in range(1, len(player_catchBallTime[i])):
        _idx = player_catchBallTime[i][j]
        _df = pd.concat([_df, Fullevents.iloc[_idx + 1:_idx + 2, :]])
    AfterPlayerCatchBall.append(_df)
fail_CatchBallRatio = []  # 接球失败率
dribbling_time = []  # 运球时间
dribbling_time_mean = []
dribbling_direction = []  # 运球方向
dribbling_direction_mean = []
passing_ratio = []  # 传球比例
for i in range(n_player):
    _df = AfterPlayerCatchBall[i]
    _df_hus = _df[_df['TeamID'] == 'Huskies']
    _df_opp = _df[_df['TeamID'] != 'Huskies']
    fail_CatchBallRatio.append(_df_opp.shape[0] / _df.shape[0])

    _df_catchball = player_catchBall[i]
    _df_catchball['dribbling_time'] = np.asfarray(_df['EventTime']) - np.asfarray(_df_catchball['EventTime'])
    _df_catchball = _df_catchball[_df_catchball['TeamID'] == 'Huskies']
    _lst = list(_df_catchball['dribbling_time'])
    for t in range(len(_lst)):
        if _lst[t] < 0:
            _lst[t] = 0
    dribbling_time.append(_lst)
    dribbling_time_mean.append(np.mean(dribbling_time[i]))

    _df_catchball = player_catchBall[i]
    _df_catchball['dribbling_direction'] = np.asfarray(_df['EventOrigin_x']) - np.asfarray(
        _df_catchball['EventOrigin_x'])
    _df_catchball = _df_catchball[_df_catchball['TeamID'] == 'Huskies']
    _lst = list(_df_catchball['dribbling_direction'])
    dribbling_direction.append(_lst)
    dribbling_direction_mean.append(np.mean(dribbling_direction[i]))

    _df_pass = _df[_df['EventType'] == 'Pass']
    passing_ratio.append(_df_pass.shape[0] / _df.shape[0])

player_attr = pd.DataFrame({'FailtoCatchBallRatio': fail_CatchBallRatio,
                            'DribblingTime_mean': dribbling_time_mean,
                            'DribblingDirc_mean': dribbling_direction_mean,
                            'Passing_ratio': passing_ratio}, index=huskies_player)
# 接球失败率绘图
_df = player_attr.sort_values(by='FailtoCatchBallRatio', ascending=False)
_ply = list(_df.index)
plt.figure(dpi=100, figsize=[10, 4])
plt.bar([i[8:] for i in _ply], _df['FailtoCatchBallRatio'])
# player_attr['FailtoCatchBallRatio'].plot('bar')
plt.show()
# 各球员平均运球时间
_df = player_attr.sort_values(by='DribblingTime_mean', ascending=False)
_ply = list(_df.index)
plt.figure(figsize=[10, 4], dpi=100)
plt.bar([i[8:] for i in _ply], _df['DribblingTime_mean'])
plt.ylabel('Dribbling time')
plt.show()
# 各球员传球比例
_df = player_attr.sort_values(by='Passing_ratio', ascending=False)
_ply = list(_df.index)
plt.figure(figsize=[10, 4], dpi=100)
plt.plot([i[8:] for i in _ply], _df['Passing_ratio'])
plt.grid()
plt.ylabel('Passing ratio')
plt.show()
# 按球员类型进行分析
Fullevents['PlayerClass'] = [i[8] for i in Fullevents['OriginPlayerID']]
forward = Fullevents[Fullevents['PlayerClass'] == 'F']
defence = Fullevents[Fullevents['PlayerClass'] == 'D']
midfield = Fullevents[Fullevents['PlayerClass'] == 'M']
goalkeeper = Fullevents[Fullevents['PlayerClass'] == 'G']

for i in range(goalkeeper.shape[0]):
    if goalkeeper['EventOrigin_x'].iloc[i] == 100:
        goalkeeper['EventOrigin_x'].iloc[i] = 0
        goalkeeper['EventOrigin_y'].iloc[i] = 0
goalkeeper_member = list(set(goalkeeper['OriginPlayerID']))
midfield_member = list(set(midfield['OriginPlayerID']))
defence_member = list(set(defence['OriginPlayerID']))
forward_member = list(set(forward['OriginPlayerID']))
'''
goalkeeper_member = [i[8] for i in goalkeeper_member]
midfield_member   = [i[8] for i in midfield_member]
defence_member    = [i[8] for i in defence_member]
forward_member    = [i[8] for i in forward_member]
'''
# 对守门员位置聚类分析 主要是近距离发球
#设置类别为2
clf_goalkeeper = KMeans(n_clusters=5)
#将数据代入到聚类模型中
clf_goalkeeper = clf_goalkeeper.fit(goalkeeper[['EventDestination_x', 'EventDestination_y']])
clf_goalkeeper.cluster_centers_
#在原始数据表中增加聚类结果标签
goalkeeper['label'] = clf_goalkeeper.labels_
_df0 = goalkeeper[goalkeeper['label'] == 0]
_df1 = goalkeeper[goalkeeper['label'] == 1]
_df2 = goalkeeper[goalkeeper['label'] == 2]
_df3 = goalkeeper[goalkeeper['label'] == 3]
_df4 = goalkeeper[goalkeeper['label'] == 4]
plt.figure()
plt.plot(_df0['EventDestination_x'], _df0['EventDestination_y'], 'r.')
plt.plot(_df1['EventDestination_x'], _df1['EventDestination_y'], 'b.')
plt.plot(_df2['EventDestination_x'], _df2['EventDestination_y'], 'y.')
plt.plot(_df3['EventDestination_x'], _df3['EventDestination_y'], 'g.')
plt.plot(_df4['EventDestination_x'], _df4['EventDestination_y'], 'm.')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Positions to which the goalkeeper kick/pass the ball')
plt.show()
# 对中场传球聚类分析
#设置类别为2
clf_midfield = KMeans(n_clusters=5)
#将数据代入到聚类模型中
clf_goalkeeper = clf_goalkeeper.fit(midfield[['EventDestination_x', 'EventDestination_y']])
clf_goalkeeper.cluster_centers_
#在原始数据表中增加聚类结果标签
goalkeeper['label'] = clf_goalkeeper.labels_
_df0 = midfield[midfield['label'] == 0]
_df1 = midfield[midfield['label'] == 1]
_df2 = midfield[midfield['label'] == 2]
_df3 = midfield[midfield['label'] == 3]
_df4 = midfield[midfield['label'] == 4]
plt.figure()
plt.plot(_df0['EventDestination_x'], _df0['EventDestination_y'], 'r.')
plt.plot(_df1['EventDestination_x'], _df1['EventDestination_y'], 'b.')
plt.plot(_df2['EventDestination_x'], _df2['EventDestination_y'], 'y.')
plt.plot(_df3['EventDestination_x'], _df3['EventDestination_y'], 'g.')
plt.plot(_df4['EventDestination_x'], _df4['EventDestination_y'], 'm.')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Positions to which the goalkeeper kick/pass the ball')
plt.show()

# 分场次球员表现
player_catchBall_df = player_catchBall[0]
AfterPlayerCatchBall_df = AfterPlayerCatchBall[0]

for i in range(n_player):
    player_catchBall_df = player_catchBall_df.append(player_catchBall[i])
    AfterPlayerCatchBall_df = AfterPlayerCatchBall_df.append(AfterPlayerCatchBall[i])

player_catchBall_byMatch = []
AfterPlayerCatchBall_byMatch = []

for i in range(n_match):
    _df = player_catchBall_df[player_catchBall_df['MatchID'] == str(i + 1)]
    player_catchBall_byMatch.append(_df)
    _df = AfterPlayerCatchBall_df[AfterPlayerCatchBall_df['MatchID'] == str(i + 1)]
    AfterPlayerCatchBall_byMatch.append(_df)

fail_CatchBallRatio_byMatch = []  # 接球失败率
dribbling_time_byMatch = []  # 运球时间
dribbling_time_mean_byMatch = []
dribbling_direction_byMatch = []  # 运球方向
dribbling_direction_mean_byMatch = []
passing_ratio_byMatch = []  # 传球比例

match_ID = list(set(Fullevents['MatchID']))
n_match = len(match_ID)
match_ID = [str(i) for i in range(1, n_match + 1)]
for i in range(n_match):
    _df = AfterPlayerCatchBall_byMatch[i]
    _df_hus = _df[_df['TeamID'] == 'Huskies']
    _df_opp = _df[_df['TeamID'] != 'Huskies']
    fail_CatchBallRatio_byMatch.append(_df_opp.shape[0] / _df.shape[0])

    _df_catchball = player_catchBall_byMatch[i]
    _df_catchball['dribbling_time'] = np.asfarray(_df['EventTime']) - np.asfarray(_df_catchball['EventTime'])
    _df_catchball = _df_catchball[_df_catchball['TeamID'] == 'Huskies']
    _lst = list(_df_catchball['dribbling_time'])
    for t in range(len(_lst)):
        if _lst[t] < 0:
            _lst[t] = 0
    _df_catchball['dribbling_time']
    dribbling_time_byMatch.append(_lst)
    dribbling_time_mean_byMatch.append(np.mean(dribbling_time_byMatch[i]))

    _df_catchball = player_catchBall_byMatch[i]
    _df_catchball['dribbling_direction'] = np.asfarray(_df['EventOrigin_x']) - np.asfarray(
        _df_catchball['EventOrigin_x'])
    _df_catchball = _df_catchball[_df_catchball['TeamID'] == 'Huskies']
    _lst = list(_df_catchball['dribbling_direction'])
    dribbling_direction_byMatch.append(_lst)
    dribbling_direction_mean_byMatch.append(np.nanmean(dribbling_direction_byMatch[i]))

    _df_pass = _df[_df['EventType'] == 'Pass']
    passing_ratio_byMatch.append(_df_pass.shape[0] / _df.shape[0])

player_attr_byMatch = pd.DataFrame({'FailtoCatchBallRatio': fail_CatchBallRatio_byMatch,
                                    'DribblingTime_mean': dribbling_time_mean_byMatch,
                                    'DribblingDirc_mean': dribbling_direction_mean_byMatch,
                                    'Passing_ratio': passing_ratio_byMatch}, index=match_ID)

# 换人策略 哈士奇每次比赛必在下半场换三人
_df = Fullevents[Fullevents['TeamID'] == 'Huskies']
substitution_hus = _df[_df['EventType'] == 'Substitution']
substitution_hus = substitution_hus.drop('PlayerClass', axis=1)
substitution_hus['OriginPlayerClass'] = [i[8] for i in substitution_hus['OriginPlayerID']]
substitution_hus['DestinationPlayerClass'] = [i[8] for i in substitution_hus['DestinationPlayerID']]
'''
forward     'F'
midfield    'M'
defence     'D'
goalkeeper  'G'
'''
# 记录换人是否使球队攻击性变强
substitution_hus_score = []
for i in range(n_match):
    _df = substitution_hus[substitution_hus['MatchID'] == str(i + 1)]
    _df = _df[['MatchID', 'OriginPlayerClass', 'DestinationPlayerClass']]
    _score = 0
    for j in range(_df.shape[0]):
        if _df['OriginPlayerClass'].iloc[j] == 'D':
            if _df['DestinationPlayerClass'].iloc[j] == 'D':
                _score += 0
            elif _df['DestinationPlayerClass'].iloc[j] == 'M':
                _score += 1
            elif _df['DestinationPlayerClass'].iloc[j] == 'F':
                _score += 2
            else:
                pass
        elif _df['OriginPlayerClass'].iloc[j] == 'M':
            if _df['DestinationPlayerClass'].iloc[j] == 'D':
                _score += -1
            elif _df['DestinationPlayerClass'].iloc[j] == 'M':
                _score += 0
            elif _df['DestinationPlayerClass'].iloc[j] == 'F':
                _score += 1
            else:
                pass
        elif _df['OriginPlayerClass'].iloc[j] == 'F':
            if _df['DestinationPlayerClass'].iloc[j] == 'D':
                _score += -2
            elif _df['DestinationPlayerClass'].iloc[j] == 'M':
                _score += -1
            elif _df['DestinationPlayerClass'].iloc[j] == 'F':
                _score += 0
            else:
                pass
        else:
            pass
    substitution_hus_score.append(_score)
substitution_hus_score = pd.DataFrame({'Substitution_AttackDefence': substitution_hus_score},
                                      index=match_ID)


KeyError: 'OwnScore'